In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import StandardScaler

rng=np.random.default_rng(7)
n=400
Tenure=rng.uniform(0,72,n)
MonthlyCharges=rng.uniform(20,120,n)
score=-0.04*Tenure+0.03*MonthlyCharges-1.0
y=(score>0).astype(int)
flip_idx=rng.choice(n,size=int(0.08*n),replace=False)
y[flip_idx]=1-y[flip_idx]
X=np.column_stack([Tenure,MonthlyCharges])
print("X shape:",X.shape,"| churn rate (mean of y):",y.mean())

def plot_points(X,y,alpha=0.85):
    plt.scatter(X[y==0,0],X[y==0,1],marker='o',alpha=alpha,label='Retained (0)',c='steelblue')
    plt.scatter(X[y==1,0],X[y==1,1],marker='s',alpha=alpha,label='Churn (1)',c='darkorange')

def plot_boundary_raw(w,b,x1_min,x1_max,label,color='crimson'):
    xs=np.linspace(x1_min,x1_max,300)
    if abs(w[1])<1e-12:
        x0=-b/(w[0]+1e-12)
        plt.axvline(x0,linewidth=2,label=label,c=color)
    else:
        ys=-(b+w[0]*xs)/w[1]
        plt.plot(xs,ys,linewidth=2,label=label,c=color)

def plot_cm(cm,labels,title):
    plt.figure(figsize=(4.4,4.0))
    plt.imshow(cm,interpolation='nearest',cmap='Blues')
    plt.title(title)
    plt.colorbar()
    ticks=np.arange(len(labels))
    plt.xticks(ticks,labels)
    plt.yticks(ticks,labels)
    thresh=cm.max()/2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j,i,str(cm[i,j]),ha='center',va='center',color='white' if cm[i,j]>thresh else 'black')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.show()

raw_clf=Perceptron(fit_intercept=True,max_iter=10,tol=None,eta0=1.0,random_state=0)
raw_clf.fit(X,y)
w_raw=raw_clf.coef_.ravel()
b_raw=raw_clf.intercept_.item()
pred_raw=raw_clf.predict(X)
acc_raw=accuracy_score(y,pred_raw)
cm_raw=confusion_matrix(y,pred_raw)
print(f"RAW  | acc={acc_raw:.3f} | w={w_raw} | b={b_raw:.3f}")
print("Confusion matrix (raw):\n",cm_raw)

plt.figure(figsize=(5.6,4.8))
plot_points(X,y)
plot_boundary_raw(w_raw,b_raw,X[:,0].min()-2,X[:,0].max()+2,"Perceptron (raw)",color='crimson')
plt.xlabel("Tenure (months)")
plt.ylabel("MonthlyCharges ($/mo)")
plt.title("Decision Boundary on RAW Features")
plt.grid(True,color='lightgray',linestyle='--',alpha=0.7)
plt.legend(loc="best")
plt.tight_layout()
plt.show()

plot_cm(cm_raw,["0","1"],"Confusion Matrix (RAW)")

scaler=StandardScaler()
X_std=scaler.fit_transform(X)
std_clf=Perceptron(fit_intercept=True,max_iter=10,tol=None,eta0=1.0,random_state=0)
std_clf.fit(X_std,y)
w_std=std_clf.coef_.ravel()
b_std=std_clf.intercept_.item()
pred_std=std_clf.predict(X_std)
acc_std=accuracy_score(y,pred_std)
cm_std=confusion_matrix(y,pred_std)
w_equiv=w_std/scaler.scale_
b_equiv=b_std-np.dot(w_std,scaler.mean_/scaler.scale_)
print(f"STD  | acc={acc_std:.3f} | w_std={w_std} | b_std={b_std:.3f}")
print("Back in original units => w_equiv=",w_equiv,"| b_equiv=",round(b_equiv,3))
print("Confusion matrix (standardized):\n",cm_std)

plt.figure(figsize=(6.2,5.0))
plot_points(X,y)
plot_boundary_raw(w_raw,b_raw,X[:,0].min()-2,X[:,0].max()+2,"Perceptron (raw)",color='crimson')
plot_boundary_raw(w_equiv,b_equiv,X[:,0].min()-2,X[:,0].max()+2,"Perceptron (standardized→raw)",color='teal')
plt.xlabel("Tenure (months)");plt.ylabel("MonthlyCharges ($/mo)")
plt.title("RAW vs STANDARDIZED (same axes): boundary clarity & stability")
plt.grid(True,color='lightgray',linestyle='--',alpha=0.7)
plt.legend(loc="best")
plt.tight_layout()
plt.show()

plot_cm(cm_std,["0","1"],"Confusion Matrix (STANDARDIZED)")
